# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to jeyjeev@gmail.com and will expire on May 21, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\jeyan\AppData\Local\Temp\graphlab_server_1533362535.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.833853     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.884227     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [10]:
tab=model_all.get("coefficients")

In [11]:
tab

name,index,value,stderr
(intercept),None,274873.05595,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [14]:
import numpy as np

In [25]:
l1_penalty_range=np.logspace(1,7,num=13)

validation_rss = {}
for l1_penalty in l1_penalty_range:
  reg_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
  predictions=reg_model.predict(training)
  residuals=training['price']-predictions
  rss = sum(residuals * residuals)
  validation_rss[l1_penalty] = rss
  #print "l1_penalty: ", l1_penalty
  #print "RSS: ", rss
print "Minimum RSS: ", min(validation_rss.items(), key=lambda x: x[1])     

l1_penalty:  10.0
RSS:  7.30724649671e+14
l1_penalty:  31.6227766017
RSS:  7.30724649991e+14
l1_penalty:  100.0
RSS:  7.30724651003e+14
l1_penalty:  316.227766017
RSS:  7.30724654205e+14
l1_penalty:  1000.0
RSS:  7.30724664327e+14
l1_penalty:  3162.27766017
RSS:  7.30724696339e+14
l1_penalty:  10000.0
RSS:  7.30724797572e+14
l1_penalty:  31622.7766017
RSS:  7.3072511774e+14
l1_penalty:  100000.0
RSS:  7.30726130614e+14
l1_penalty:  316227.766017
RSS:  7.30729337734e+14
l1_penalty:  1000000.0
RSS:  7.30739520853e+14
l1_penalty:  3162277.66017
RSS:  7.3077213586e+14
l1_penalty:  10000000.0
RSS:  7.30884676668e+14
Minimum RSS:  (10.0, 730724649671115.6)


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [30]:
model_test= graphlab.linear_regression.create(testing, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0., l1_penalty=10,verbose=False)
predictions_test=model_test.predict(testing)
residuals_test=testing['price']-predictions_test
rss_test = sum(residuals_test * residuals_test)
print "RSS Test ", rss_test

RSS Test  1.55205417245e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [31]:
non_zero_weight_test = model_test["coefficients"][model_test["coefficients"]["value"] > 0]
print model_test["coefficients"]["value"].nnz()
non_zero_weight_test.print_rows(num_rows=20)

18
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 15938.3491302 |  None  |
|     bedrooms     |  None | 7927.96151068 |  None  |
| bedrooms_square  |  None |  2811.6551531 |  None  |
|    bathrooms     |  None |  25090.812508 |  None  |
|   sqft_living    |  None | 39.6894164796 |  None  |
| sqft_living_sqrt |  None | 1115.10438309 |  None  |
|  sqft_lot_sqrt   |  None | 126.735040031 |  None  |
|      floors      |  None | 19806.9697903 |  None  |
|  floors_square   |  None | 12646.6371612 |  None  |
|    waterfront    |  None | 413704.286579 |  None  |
|       view       |  None | 86807.2319587 |  None  |
|    condition     |  None | 5914.49546888 |  None  |
|      grade       |  None | 6017.72238936 |  None  |
|    sqft_above    |  None | 44.5451590346 |  None  |
|  sqft_basement   |  None |  109.73040017 |  None  |
|     yr_built     |  Non

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [34]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [35]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [73]:
l1_penalty_min_arr ={}
l1_penalty_max_arr ={}

i=0
j=0
for l1_penalty in l1_penalty_values:
  model_train= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
  #non_zero_weight_train = model_train["coefficients"][model_train["coefficients"]["value"] > 0]
  non_zero_weight_train=model_train["coefficients"]["value"].nnz()
  #non_zero_weight_test.print_rows(num_rows=20)
  
 # print non_zero_weight_train, l1_penalty
  if non_zero_weight_train > max_nonzeros:
       l1_penalty_min_arr[i]=l1_penalty
       i= i+1
  elif non_zero_weight_train < max_nonzeros:
       l1_penalty_max_arr[j]=l1_penalty
       j= j+1

l1_penalty_min=max(l1_penalty_min_arr.items(), key=lambda x: x[1])
l1_penalty_max=min(l1_penalty_max_arr.items(), key=lambda x: x[1])
print "Largest L1 Penalty: ", l1_penalty_min
print "Smallest L1 Penalty: ", l1_penalty_max

Largest L1 Penalty:  (10, 3406003414.8423343)
Smallest L1 Penalty:  (0, 3620829401.447845)


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [76]:
l1_penalty_min =  2976351441.6313128

l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [77]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [78]:
validation_rss={}
for l1_penalty in l1_penalty_values:
  reg_model_narrow = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None,l2_penalty=0., l1_penalty=l1_penalty,verbose=False)
  predictions=reg_model_narrow.predict(training)
  residuals=training['price']-predictions
  rss = sum(residuals * residuals)
  validation_rss[l1_penalty] = rss, reg_model_narrow['coefficients']['value'].nnz()
  #print "RSS: ", validation_rss 
#print "Minimum RSS: ", min(validation_rss.items(), key=lambda x: x[1])   

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [84]:
print "RSS: ", validation_rss
for k,v in validation_rss.iteritems():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]=0
        bestl1 = k
        
print bestRSS, bestl1

RSS:  {3749724993.4111514: (1240410027717370.2, 6), 3148212230.9157214: (1153325671277096.0, 10), 3363038217.5212321: (1197205535540331.5, 8), 3577864204.1267428: (1221756730455491.0, 7), 3792690190.7322536: (1245103053330360.0, 6), 2976351441.6313128: (1118068724936751.9, 10), 3191177428.2368236: (1164002080668110.2, 10), 3406003414.8423343: (1202351963326026.0, 8), 3620829401.447845: (1227165195287587.8, 6), 3019316638.952415: (1125944839197887.0, 10), 3234142625.5579257: (1174814612312642.0, 10), 3448968612.1634364: (1206458853787676.0, 7), 3663794598.7689471: (1231345655244412.8, 6), 3062281836.2735171: (1133902111838455.5, 10), 3277107822.8790278: (1185763200569767.8, 10), 3491933809.4845386: (1211117632036090.2, 7), 3706759796.0900493: (1235830828341440.2, 6), 3105247033.5946193: (1142905527629250.8, 10), 3320073020.20013: (1192858597198544.0, 8), 3534899006.8056407: (1216458020728344.0, 7)}
0 0


In [81]:
for k,v in validation_rss.iteritems():
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        print k, bestRSS

In [82]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)

In [83]:
non_zero_weight_test = model["coefficients"][model["coefficients"]["value"] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

